In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() \
                      else "cpu")
device = "cpu"
print("working on: " + str(device))

working on: cpu


In [3]:
df = pd.read_csv("pmsm_temperature_data.csv")

In [4]:
df_sep = [df[df.profile_id==profile].drop(['profile_id','torque'], axis=1).reset_index(drop=True) for profile in df.profile_id.unique()]

train_min = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque'], axis=1).min()
train_max = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque'], axis=1).max()

df_train = [(df_sep[i] - train_min)/(train_max - train_min)\
            for i in list(range(0,52)) if i not in [4, 8, 16, 24, 32, 
                                                    40, 48, 51, #val
                                                    35, 42]] #test

df_val = [(df_sep[i] - train_min)/(train_max - train_min)\
          for i in [4, 8, 16, 24, 32, 40, 48, 51]]

df_test = [(df_sep[i] - train_min)/(train_max - train_min) for i in [35, 42]]

In [5]:
def dataloader(data, length, shuffle=True, out=1):
    while True:
    # genera una lista di (i_serie, i_obs)
        tuples = [[(df_i, i) for i, x in enumerate(data[df_i]) if i >= length + out]
                  for df_i, _ in enumerate(data)]
        tuples = sum(tuples, [])  # flattenizza
        # shuffle
        if shuffle:
            np.random.shuffle(tuples)

        # yielda le osservazioni
        for df_i, i in tuples:
            X_lagged = data[df_i][(i - length - out):(i - out), FEATURES + TARGET]
            y = data[df_i][(i-out):(i), TARGET]
            yield X_lagged, y

In [6]:
class Custom_RNN(torch.nn.Module):

    def __init__(self, hiddens, recur,
                 regressors,
                 p_dropout=0.3):
        self.recur_input, self.recur_output = recur
        # recur_input in [0, len(hiddens))
        # recur_output in [-1, len(hiddens)]
        assert 0 <= self.recur_input <= len(hiddens)
        assert -1 <= self.recur_output < len(hiddens)
        assert len(regressors) == 2

        super(Custom_RNN, self).__init__()
        # input_shape = len(FEATURES)  # new task
        input_shape = len(FEATURES) + len(TARGET)  # old task
        self.previous_size = hiddens[self.recur_output] \
            if self.recur_output >= 0 else len(TARGET)
        
        hiddens_in = [input_shape] + hiddens

        hiddens_in[self.recur_input] += self.previous_size

        self.dropout = torch.nn.Dropout(p=p_dropout)
        self.layers = torch.nn.ModuleList(
            [torch.nn.Linear(in_shape, out_shape) \
             for in_shape, out_shape in zip(hiddens_in, hiddens)])
        self.layers_fn = [torch.nn.functional.relu \
                          for _ in self.layers]

        regressor_list = []
        for features, regressor in zip([1, 3], regressors):
            if type(regressor) != list:
                regressor = [regressor]
            input_shape = [hiddens[-1]] + regressor
            y_net = [torch.nn.ReLU() for _ in input_shape + regressor]
            y_net[::2] = [torch.nn.Linear(in_shape, out_shape)
                          for in_shape, out_shape
                          in zip(input_shape, regressor + [features])]
            regressor_list.append(torch.nn.Sequential(*y_net))
        self.regressors = torch.nn.ModuleList(regressor_list)

    def initialize_prev(self, x):
        return torch.zeros(len(x), self.previous_size).to(DEVICE)

    def forward(self, data, lag=None):
        _, l, _ = data.shape
        x = data[:, 0, :]  # .reshape(-1, len(FEATURES))
        if lag is None:  # first recursion
            lag = self.initialize_prev(x)
        for i, (layer, fn) in enumerate(zip(self.layers, self.layers_fn)):
            if i == self.recur_input:
                x = torch.cat([x, lag], axis=1)
            x = fn(layer(x))
            x = self.dropout(x)
            if i == self.recur_output and l > 1:
                return self.forward(data[:, 1:, :], x)
        x = torch.cat([regressor(x) for regressor in self.regressors],
                      axis=1)
        if self.recur_output == -1 and l > 1:
            return self.forward(data[:, 1:, :], x)
        return x

In [7]:
def outputSize(in_size, kernel_size, stride, padding):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)

class CNN_Net_2(nn.Module):
    def __init__(self, batch, in_c, out, 
                 filtro1, filtro2,
                 num_ser, 
                 neuroni1, neuroni2,
                 kernel1, kernel2, kernel3, kernel4, 
                 #padding1, padding2, padding3, padding4, 
                 stride1, stride2, stride3, stride4):
        super(CNN_Net_2, self).__init__()
        self.batch_size = 1
        self.in_c = in_c
        l0 = num_ser
        l1 = outputSize(l0,kernel1,stride1,0)
        l2 = outputSize(l1,kernel2,stride2,0)
        l3 = outputSize(l2,kernel3,stride3,0)
        l4 = outputSize(l3,kernel4,stride4,0)
        #print(l4)

        self.conv1 = nn.Conv1d(in_channels= in_c, 
                               out_channels= filtro1, 
                               kernel_size= kernel1,
                               stride= stride1,
                               padding=0)    
        self.pool1 = nn.MaxPool1d(kernel2,stride=stride2,padding=0) 
        self.conv2 = nn.Conv1d(filtro1,filtro2,kernel3,stride=stride3,padding=0)
        self.pool2 = nn.MaxPool1d(kernel4,stride=stride4,padding=0)
        self.fc1 = nn.Linear(filtro2*l4, neuroni1)
        self.fc2 = nn.Linear(neuroni1, neuroni2)
        self.fc3 = nn.Linear(neuroni2, out)

    def forward(self, x):
        batch_size, _, _ = x.shape
        x =  self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(batch_size, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        return  self.fc3(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:       # Get the products
            num_features *= s
        return num_features

In [8]:
best_mse = torch.load("models/best_model_cnn").to(device)
best_mse_stator = torch.load("models/best_model_cnn_stator").to(device)

/home/pranav/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:134: UserWarning: 
    Found GPU0 GeForce GT 750M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [9]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

In [10]:
look_back  = 60

train_gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(train_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


In [11]:
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

In [12]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

In [13]:
look_back  = 60

train_gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(train_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


In [15]:
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2

In [18]:
np.mean(np.concatenate((pm_err, pm_err_stat)))

2.3169167135008078e-05

In [ ]:
#validation

In [26]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

val_gen = dataloader([df.values for df in df_val], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_val)


batch_size = 7403

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(val_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
7403
14806
22209
29612
37015
44418
51821
59224
66627
74030
81433
88836
96239
103642
111045
118448
125851
133254
140657
148060
155463
162866


In [27]:
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

In [31]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

look_back  = 60

val_gen = dataloader([df.values for df in df_val], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_val)


batch_size = 7403

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(val_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
7403
14806
22209
29612
37015
44418
51821
59224
66627
74030
81433
88836
96239
103642
111045
118448
125851
133254
140657
148060
155463
162866


In [32]:
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2

In [33]:
np.mean(np.concatenate((pm_err, pm_err_stat)))

2.2815047581673798e-05

In [34]:
#test

In [36]:
size

56878

In [9]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

test_gen = dataloader([df.values for df in df_test], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_test)


batch_size = 28439

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(test_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

look_back  = 60

test_gen = dataloader([df.values for df in df_test], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_test)


batch_size = 28439

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(test_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
28439


/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0
28439


2.3870404711804732e-05

# LSTM

In [11]:
class LSTM_Net(nn.Module):
    def __init__(self, features, hidden, h1, h2, out):
        super(LSTM_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.lstm = nn.LSTM(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.lstm(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [12]:
best_mse = torch.load("models/best_model_lstm").to(device)
best_mse_stator = torch.load("models/best_model_lstm_stator").to(device)

In [13]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

look_back  = 60

gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0


/home/pranav/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1347: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


9.836819436559445e-06

In [14]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_val], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_val)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

gen = dataloader([df.values for df in df_val], look_back, shuffle=False)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
0
18803
37606
56409
75212
94015
112818
131621
150424
169227


9.54949601349374e-06

In [15]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_test], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_test)


batch_size = 28439

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

gen = dataloader([df.values for df in df_test], look_back, shuffle=False)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
28439
0
28439


1.009048952520101e-05

# GRU

In [16]:
class GRU_Net(nn.Module):
    def __init__(self, features, hidden, h1, h2, out):
        super(GRU_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.gru = nn.GRU(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.gru(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [17]:
best_mse = torch.load("models/best_model_gru").to(device)
best_mse_stator = torch.load("models/best_model_gru_stator").to(device)

In [18]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

look_back  = 60

gen = dataloader([df.values for df in df_train], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_train)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
188030
206833
225636
244439
263242
282045
300848
319651
338454
357257
376060
394863
413666
432469
451272
470075
488878
507681
526484
545287
564090
582893
601696
620499
639302
658105
676908
695711
714514
733317
752120


1.2924898004638905e-05

In [19]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_val], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_val)


batch_size = 18803

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

gen = dataloader([df.values for df in df_val], look_back, shuffle=False)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
18803
37606
56409
75212
94015
112818
131621
150424
169227
0
18803
37606
56409
75212
94015
112818
131621
150424
169227


1.2424951524082883e-05

In [20]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

look_back  = 60

gen = dataloader([df.values for df in df_test], look_back, shuffle=False)
size = np.sum(df.shape[0] for df in df_test)


batch_size = 28439

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 8)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err = (y_pred_all - np.array(y_test).reshape(-1))**2

FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [8, 9, 10]

gen = dataloader([df.values for df in df_test], look_back, shuffle=False)

y_test = []
y_pred_all = []
for x in range(0, size, batch_size):
    print(x)
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break
    inpt = np.array(X_test).reshape(-1, look_back, 10)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_mse_stator(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())
    
    
pm_err_stat = (y_pred_all.reshape(-1) - np.array(y_test).reshape(-1))**2


np.mean(np.concatenate((pm_err, pm_err_stat)))

/home/pranav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  import sys


0
28439
0
28439


1.2822127989885529e-05